In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
train_data = pd.read_csv('../data/archive/cross-val/mouth/mouthLabels.csv')

In [ ]:
# cropped mouth
train_data = pd.read_csv('../data/archive/cross-val/mouth-cropped/mouthLabels.csv')

In [ ]:
Y = train_data[['label']]

In [ ]:
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

In [ ]:
idg  = ImageDataGenerator(rescale=1./255)

In [ ]:
#model = keras.models.load_model("base_yawning_model.h5")
#model = keras.models.load_model("yawning_model.h5")
#model = keras.models.load_model("mobilenet_yawning_model.h5")
#model = keras.models.load_model("base_cropped_yawning_model.h5")
#model = keras.models.load_model("inception_cropped_yawning_model.h5")
model = keras.models.load_model("mobileV2_cropped_yawning_model.h5")

In [ ]:
model.summary()

In [ ]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

for train_index, val_index in skf.split(np.zeros(len(Y)),Y):
	validation_data = train_data.iloc[val_index]
	
	# cropped mouth is (80, 80) else (224, 224) 
	valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = '../data/archive/', target_size=(80, 80),
							x_col = "filename", y_col = "label",
							class_mode = "binary", shuffle = True)
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])

In [ ]:
np.mean(VALIDATION_ACCURACY) # base cropped yawning_model

In [ ]:
np.mean(VALIDATION_ACCURACY) # inception cropped yawning_model

In [ ]:
np.mean(VALIDATION_ACCURACY) # mobile cropped yawning_model

In [ ]:
np.mean(VALIDATION_ACCURACY) # base yawning_model

In [ ]:
np.mean(VALIDATION_ACCURACY) # inceptionresnetv2 yawning_model

In [ ]:
np.mean(VALIDATION_ACCURACY) # mobilenet_yawning_model